In [1]:
# ! pip install pycoingecko ccxt

In [2]:
# """
# Ethereum Price Prediction - Data Collection Module
# 수집 데이터:
# - Cryptocurrency OHLCV (yfinance + CCXT)
# - Macro indicators (SP500, VIX, Gold, DXY)
# - Fear & Greed Index
# - DeFi TVL (Protocols, L2 chains)
# - ETH Funding Rate
# """

# import os
# import time
# import warnings
# from datetime import datetime, timedelta

# import pandas as pd
# import requests
# import yfinance as yf
# import ccxt
# from binance.client import Client
# from defillama2 import DefiLlama

# warnings.filterwarnings('ignore')

# pyupbit_url = "https://api.upbit.com/v1/candles/days"
# # ============================================================================
# # Configuration
# # ============================================================================
# START_DATE = "2017-01-01"
# END_DATE = datetime.today().strftime('%Y-%m-%d')
# OUTPUT_DIR = "./macro_data"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# CRYPTO_TICKERS = {
#     'BTC-USD': ('BTC', 'BTC/USDT'),
#     'ETH-USD': ('ETH', 'ETH/USDT'),
#     'BNB-USD': ('BNB', 'BNB/USDT'),
#     'XRP-USD': ('XRP', 'XRP/USDT'),
#     'SOL-USD': ('SOL', 'SOL/USDT'),
#     'ADA-USD': ('ADA', 'ADA/USDT'),
#     'DOGE-USD': ('DOGE', 'DOGE/USDT'),
#     'AVAX-USD': ('AVAX', 'AVAX/USDT'),
#     'DOT-USD': ('DOT', 'DOT/USDT')
# }

# MACRO_TICKERS = {
#     'DX-Y.NYB': 'DXY',
#     'GC=F': 'GOLD',
#     '^VIX': 'VIX',
#     '^GSPC': 'SP500'
# }

# DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']
# L2_CHAINS = ['Arbitrum', 'Optimism', 'Base', 'zkSync Era']


# # ============================================================================
# # 1. Cryptocurrency Price Data (yfinance + CCXT)
# # ============================================================================
# def collect_crypto_prices():
#     """암호화폐 가격 데이터 수집 (yfinance 우선, CCXT 보완)"""
#     print(f"\n[1/6] Collecting cryptocurrency prices...")
    
#     start_dt = pd.to_datetime(START_DATE)
#     end_dt = pd.Timestamp.today()
#     date_range = pd.date_range(start=start_dt, end=end_dt, freq='D')
#     merged_df = pd.DataFrame(date_range, columns=['date'])
    
#     for ticker, (symbol, ccxt_symbol) in CRYPTO_TICKERS.items():
#         try:
#             df = yf.Ticker(ticker).history(start=START_DATE)
#             if df.empty:
#                 print(f"  - {symbol}: No data")
#                 continue
                
#             df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
#             df.columns = [f"{symbol}_{col}" for col in df.columns]
#             df = df.reset_index()
#             df['date'] = df['Date'].dt.tz_localize(None)
#             df = df.drop(columns=['Date'])
            
#             merged_df = pd.merge(merged_df, df, on='date', how='outer')
#             print(f"  - {symbol}: {len(df)} days")
#         except Exception as e:
#             print(f"  - {symbol}: Failed ({str(e)[:50]})")
    
#     merged_df = merged_df.sort_values('date')
#     output_path = os.path.join(OUTPUT_DIR, "macro_crypto_data.csv")
#     merged_df.to_csv(output_path, index=False)
#     print(f"  Saved: {output_path}")
#     return merged_df


# # ============================================================================
# # 2. Macro Indicators (SP500, VIX, Gold, DXY)
# # ============================================================================
# def collect_macro_indicators():
#     """거시경제 지표 수집"""
#     print(f"\n[2/6] Collecting macro indicators...")
    
#     for ticker, name in MACRO_TICKERS.items():
#         try:
#             df = yf.download(ticker, start=START_DATE, end=END_DATE, 
#                            progress=False, group_by='column')
#             df = df[['Close']].copy()
#             df.columns = [name]
#             df.index.name = 'date'
            
#             output_path = os.path.join(OUTPUT_DIR, f"{name}.csv")
#             df.to_csv(output_path)
#             print(f"  - {name}: {len(df)} days")
#         except Exception as e:
#             print(f"  - {name}: Failed ({str(e)[:50]})")


# # ============================================================================
# # 3. Fear & Greed Index
# # ============================================================================
# def collect_fear_greed():
#     """Fear & Greed Index 수집"""
#     print(f"\n[3/6] Collecting Fear & Greed Index...")
    
#     try:
#         url = "https://api.alternative.me/fng/?limit=4000&format=json"
#         response = requests.get(url)
#         data = response.json()['data']
        
#         df = pd.DataFrame(data)
#         df['date'] = pd.to_datetime(df['timestamp'], unit='s')
#         df = df[['date', 'value']].rename(columns={'value': 'fear_greed'})
#         df = df.sort_values('date').reset_index(drop=True)
        
#         output_path = os.path.join(OUTPUT_DIR, "fear_greed.csv")
#         df.to_csv(output_path, index=False)
#         print(f"  - Fear & Greed: {len(df)} days")
#     except Exception as e:
#         print(f"  - Fear & Greed: Failed ({str(e)[:50]})")


# # ============================================================================
# # 4. ETH Funding Rate
# # ============================================================================
# def collect_funding_rate():
#     """이더리움 펀딩비 수집"""
#     print(f"\n[4/6] Collecting ETH funding rate...")
    
#     try:
#         client = Client("", "")
#         funding_rates = []
        
#         start_time = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
#         end_time = int(datetime.strptime(END_DATE, "%Y-%m-%d").timestamp() * 1000)
#         current_ts = start_time
        
#         while current_ts < end_time:
#             rates = client.futures_funding_rate(
#                 symbol='ETHUSDT',
#                 startTime=current_ts,
#                 limit=1000
#             )
#             if not rates:
#                 break
#             funding_rates.extend(rates)
#             current_ts = rates[-1]['fundingTime'] + 1
        
#         df = pd.DataFrame(funding_rates)
#         df['date'] = pd.to_datetime(df['fundingTime'], unit='ms').dt.date
#         df['fundingRate'] = df['fundingRate'].astype(float)
#         df = df.groupby('date')['fundingRate'].mean().reset_index()
        
#         output_path = os.path.join(OUTPUT_DIR, "eth_funding_rate.csv")
#         df.to_csv(output_path, index=False)
#         print(f"  - Funding Rate: {len(df)} days")
#     except Exception as e:
#         print(f"  - Funding Rate: Failed ({str(e)[:50]})")


# # ============================================================================
# # 5. DeFi Protocol TVL
# # ============================================================================
# def collect_defi_tvl():
#     """DeFi 프로토콜 TVL 수집"""
#     print(f"\n[5/6] Collecting DeFi protocol TVL...")
    
#     obj = DefiLlama()
    
#     # Ethereum Chain TVL
#     try:
#         df = obj.get_chain_hist_tvl('Ethereum')
#         df = df.reset_index().rename(columns={'tvl': 'eth_chain_tvl'})
#         output_path = os.path.join(OUTPUT_DIR, 'eth_chain_tvl.csv')
#         df.to_csv(output_path, index=False)
#         print(f"  - ETH Chain TVL: {len(df)} days")
#     except Exception as e:
#         print(f"  - ETH Chain TVL: Failed ({str(e)[:50]})")
    
#     # Protocol TVL (Ethereum only)
#     for protocol in DEFI_PROTOCOLS:
#         try:
#             tvl_dict = obj.get_protocol_hist_tvl_by_chain(protocol)
#             if 'Ethereum' in tvl_dict:
#                 df = tvl_dict['Ethereum'].reset_index()
#                 df = df.rename(columns={'tvl': f'{protocol}_eth_tvl'})
#                 output_path = os.path.join(OUTPUT_DIR, f'{protocol}_eth_tvl.csv')
#                 df.to_csv(output_path, index=False)
#                 print(f"  - {protocol}: {len(df)} days")
#             else:
#                 print(f"  - {protocol}: No Ethereum data")
#         except Exception as e:
#             print(f"  - {protocol}: Failed ({str(e)[:50]})")
    
#     # USDT Market Cap
#     try:
#         df = obj.get_stablecoin_hist_mcap_on_a_chain(1, 'ethereum')
#         df = df.reset_index().rename(columns={'mcap': 'usdt_eth_mcap'})
#         output_path = os.path.join(OUTPUT_DIR, 'usdt_eth_mcap.csv')
#         df.to_csv(output_path, index=False)
#         print(f"  - USDT ETH Mcap: {len(df)} days")
#     except Exception as e:
#         print(f"  - USDT ETH Mcap: Failed ({str(e)[:50]})")


# # ============================================================================
# # 6. Layer 2 TVL
# # ============================================================================
# def collect_l2_tvl():
#     """Layer 2 TVL 수집"""
#     print(f"\n[6/6] Collecting Layer 2 TVL...")
    
#     all_data = []
    
#     for chain in L2_CHAINS:
#         try:
#             url = f"https://api.llama.fi/v2/historicalChainTvl/{chain}"
#             response = requests.get(url)
            
#             if response.status_code == 200:
#                 data = response.json()
#                 df = pd.DataFrame(data)
#                 df['date'] = pd.to_datetime(df['date'], unit='s')
#                 df = df.rename(columns={'tvl': f'{chain.lower()}_tvl'})
#                 all_data.append(df[['date', f'{chain.lower()}_tvl']])
#                 print(f"  - {chain}: {len(df)} days")
#             else:
#                 print(f"  - {chain}: API error")
#         except Exception as e:
#             print(f"  - {chain}: Failed ({str(e)[:50]})")
    
#     if all_data:
#         result = all_data[0]
#         for df in all_data[1:]:
#             result = pd.merge(result, df, on='date', how='outer')
        
#         result = result.sort_values('date').reset_index(drop=True)
#         output_path = os.path.join(OUTPUT_DIR, 'layer2_tvl.csv')
#         result.to_csv(output_path, index=False)
#         print(f"  Saved: {output_path}")


# # ============================================================================
# # Main Execution
# # ============================================================================
# if __name__ == "__main__":
#     print("=" * 80)
#     print("ETH Price Prediction - Data Collection Pipeline")
#     print("=" * 80)
    
#     collect_crypto_prices()
#     collect_macro_indicators()
#     collect_fear_greed()
#     collect_funding_rate()
#     collect_defi_tvl()
#     collect_l2_tvl()
    
#     print("\n" + "=" * 80)
#     print("Data collection completed!")
#     print(f"Output directory: {OUTPUT_DIR}")
#     print("=" * 80)




In [3]:
import os
import time
import warnings
from datetime import datetime, timedelta
import re
import pandas as pd
import requests
import yfinance as yf
import ccxt
from binance.client import Client
from defillama2 import DefiLlama
NEWS_DIR = '../news_data'

####################################################
# 1. 데이터 로드 함수
####################################################

def parse_date_from_filename(filename):
    """파일명에서 날짜 추출"""
    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',
        r'(\d{4})(\d{2})(\d{2})',
        r'(\d{2})-(\d{2})-(\d{4})',
        r'(\d{2})(\d{2})(\d{4})'
    ]
    basename = os.path.basename(filename)
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            try:
                if len(match.group(1)) == 4:
                    year, month, day = match.groups()
                else:
                    day, month, year = match.groups()
                return pd.to_datetime(f"{year}-{month}-{day}")
            except:
                continue
    return None

def load_all_news_data(root_dir):
    """뉴스 데이터 로드 및 일간 집계"""
    print("\n=== 뉴스 데이터 로드 ===")
    
    all_data = []
    
    if not os.path.exists(root_dir):
        print(f"경고: {root_dir} 디렉토리가 존재하지 않습니다. 더미 데이터를 생성합니다.")
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    
    csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('.csv')])
    print(f"발견된 뉴스 파일: {len(csv_files)}개")
    
    for filename in csv_files:
        filepath = os.path.join(root_dir, filename)
        file_date = parse_date_from_filename(filename)
        
        # 여러 인코딩 시도
        for enc in ['utf-8', 'cp949', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=enc)
                break
            except Exception:
                continue
        else:
            print(f"  건너뜀: {filename} (인코딩 실패)")
            continue
        
        # 날짜 컬럼 처리
        if 'date' not in df.columns:
            df['date'] = file_date
        else:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            if file_date is not None:
                df['date'] = df['date'].fillna(file_date)
        
        # label 컬럼 확인
        if 'label' not in df.columns:
            print(f"  경고: {filename}에 'label' 컬럼이 없습니다. 0으로 채웁니다.")
            df['label'] = 0
        
        # 필요한 컬럼만 선택
        if 'news' in df.columns:
            df = df[['date', 'news', 'label']]
        else:
            df = df[['date', 'label']]
        
        all_data.append(df)
    
    if len(all_data) == 0:
        print("경고: 유효한 뉴스 파일을 찾을 수 없습니다. 더미 데이터를 생성합니다.")
        dates = pd.date_range(START_TIME, END_TIME, freq='D')
        return pd.DataFrame({
            'date': dates,
            'news': ['test news'] * len(dates),
            'label': np.random.choice([1, 0, -1], len(dates))
        })
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df


In [4]:
df=load_all_news_data(NEWS_DIR)
df.to_csv("news_data.csv",index=False)

df.tail()


=== 뉴스 데이터 로드 ===
발견된 뉴스 파일: 2135개


,date,news,label
26341,2025-11-05,익명 고래 트레이더가 자오창펑(CZ) 관련 발언 직후 알트코인 공매도 포지션으로 대...,0
26342,2025-11-05,마치 빅 브라더(Machi Big Brother)로 알려진 유명 트레이더 제프리 황...,-1
26343,2025-11-05,리플(Ripple)의 스테이블코인 RLUSD가 출시 1년 만에 시가총액 10억달러를...,1
26344,2025-11-05,"시바이누(Shiba Inu, SHIB)가 미국에서 현물 상장지수펀드(ETF) 출시 ...",1
26345,2025-11-05,엑스알피(XRP)와 리플의 스테이블코인 RLUSD가 글로벌 결제 혁신에 속도를 내며...,1


In [7]:
import os
import time
import warnings
from datetime import datetime, timedelta
import os
import time
import warnings

import pandas as pd
import requests
import yfinance as yf
import ccxt
from binance.client import Client
from defillama2 import DefiLlama
import pandas as pd
import requests
import yfinance as yf
from defillama2 import DefiLlama

warnings.filterwarnings('ignore')

START_DATE = "2017-01-01"
END_DATE = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
OUTPUT_DIR = "./macro_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

UPBIT_CRYPTO_TICKERS = {
    'KRW-BTC': ('BTC', 'BTC'),
    'KRW-ETH': ('ETH', 'ETH'),

    'KRW-XRP': ('XRP', 'XRP'),
    'KRW-SOL': ('SOL', 'SOL'),
    'KRW-ADA': ('ADA', 'ADA'),
    'KRW-DOGE': ('DOGE', 'DOGE'),
    'KRW-AVAX': ('AVAX', 'AVAX'),
    'KRW-DOT': ('DOT', 'DOT')
}

MACRO_TICKERS = {
    'DX-Y.NYB': 'DXY',
    'GC=F': 'GOLD',
    '^VIX': 'VIX',
    '^GSPC': 'SP500'
}

DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']
L2_CHAINS = ['Arbitrum', 'Optimism', 'Base', 'zkSync Era']


def collect_upbit_crypto_prices():
    print(f"\n[1/6] Collecting cryptocurrency prices from Upbit...")
    
    start_dt = pd.to_datetime(START_DATE)
    merged_df = None
    
    for market, (symbol, _) in UPBIT_CRYPTO_TICKERS.items():
        try:
            all_candles = []
            to_date = None
            
            while True:
                url = f"https://api.upbit.com/v1/candles/days"
                params = {
                    'market': market,
                    'count': 200
                }
                if to_date:
                    params['to'] = to_date
                
                response = requests.get(url, params=params)
                response.raise_for_status()
                candles = response.json()
                
                if not candles:
                    break
                
                all_candles.extend(candles)
                oldest_date = pd.to_datetime(candles[-1]['candle_date_time_kst'])
                
                if oldest_date <= start_dt:
                    break
                
                to_date = candles[-1]['candle_date_time_utc']
                time.sleep(0.1)
            
            if not all_candles:
                print(f"  - {symbol}: No data")
                continue
            
            df = pd.DataFrame(all_candles)
            df['date'] = pd.to_datetime(df['candle_date_time_kst']).dt.normalize()
            
            df = df.rename(columns={
                'opening_price': f'{symbol}_Open',
                'high_price': f'{symbol}_High',
                'low_price': f'{symbol}_Low',
                'trade_price': f'{symbol}_Close',
                'candle_acc_trade_volume': f'{symbol}_Volume'
            })
            
            df = df[['date', f'{symbol}_Open', f'{symbol}_High', 
                    f'{symbol}_Low', f'{symbol}_Close', f'{symbol}_Volume']]
            df = df.sort_values('date')
            df = df[df['date'] >= start_dt]
            
            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on='date', how='outer')
            
            print(f"  - {symbol}: {len(df)} days")
            
        except Exception as e:
            print(f"  - {symbol}: Failed ({str(e)[:50]})")
    
    if merged_df is not None:
        merged_df = merged_df.sort_values('date')
        output_path = os.path.join(OUTPUT_DIR, "macro_crypto_data.csv")
        merged_df.to_csv(output_path, index=False)
        print(f"  Saved: {output_path}")
    
    return merged_df

# ============================================================================
# 2. Macro Indicators (SP500, VIX, Gold, DXY)
# ============================================================================
def collect_macro_indicators():
    """거시경제 지표 수집"""
    print(f"\n[2/6] Collecting macro indicators...")
    
    for ticker, name in MACRO_TICKERS.items():
        try:
            df = yf.download(ticker, start=START_DATE, end=END_DATE, 
                           progress=False, group_by='column')
            df = df[['Close']].copy()
            df.columns = [name]
            df.index.name = 'date'
            
            output_path = os.path.join(OUTPUT_DIR, f"{name}.csv")
            df.to_csv(output_path)
            print(f"  - {name}: {len(df)} days")
        except Exception as e:
            print(f"  - {name}: Failed ({str(e)[:50]})")


# ============================================================================
# 3. Fear & Greed Index
# ============================================================================
def collect_fear_greed():
    """Fear & Greed Index 수집"""
    print(f"\n[3/6] Collecting Fear & Greed Index...")
    
    try:
        url = "https://api.alternative.me/fng/?limit=4000&format=json"
        response = requests.get(url)
        data = response.json()['data']
        
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df[['date', 'value']].rename(columns={'value': 'fear_greed'})
        df = df.sort_values('date').reset_index(drop=True)
        
        output_path = os.path.join(OUTPUT_DIR, "fear_greed.csv")
        df.to_csv(output_path, index=False)
        print(f"  - Fear & Greed: {len(df)} days")
    except Exception as e:
        print(f"  - Fear & Greed: Failed ({str(e)[:50]})")


# ============================================================================
# 4. ETH Funding Rate
# ============================================================================
def collect_funding_rate():
    """이더리움 펀딩비 수집"""
    print(f"\n[4/6] Collecting ETH funding rate...")
    
    try:
        client = Client("", "")
        funding_rates = []
        
        start_time = int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000)
        end_time = int(datetime.strptime(END_DATE, "%Y-%m-%d").timestamp() * 1000)
        current_ts = start_time
        
        while current_ts < end_time:
            rates = client.futures_funding_rate(
                symbol='ETHUSDT',
                startTime=current_ts,
                limit=1000
            )
            if not rates:
                break
            funding_rates.extend(rates)
            current_ts = rates[-1]['fundingTime'] + 1
        
        df = pd.DataFrame(funding_rates)
        df['date'] = pd.to_datetime(df['fundingTime'], unit='ms').dt.date
        df['fundingRate'] = df['fundingRate'].astype(float)
        df = df.groupby('date')['fundingRate'].mean().reset_index()
        
        output_path = os.path.join(OUTPUT_DIR, "eth_funding_rate.csv")
        df.to_csv(output_path, index=False)
        print(f"  - Funding Rate: {len(df)} days")
    except Exception as e:
        print(f"  - Funding Rate: Failed ({str(e)[:50]})")


# ============================================================================
# 5. DeFi Protocol TVL
# ============================================================================
def collect_defi_tvl():
    """DeFi 프로토콜 TVL 수집"""
    print(f"\n[5/6] Collecting DeFi protocol TVL...")
    
    obj = DefiLlama()
    
#     # Ethereum Chain TVL
    try:
        df = obj.get_chain_hist_tvl('Ethereum')
        df = df.reset_index().rename(columns={'tvl': 'eth_chain_tvl'})
        output_path = os.path.join(OUTPUT_DIR, 'eth_chain_tvl.csv')
        df.to_csv(output_path, index=False)
        print(f"  - ETH Chain TVL: {len(df)} days")
    except Exception as e:
        print(f"  - ETH Chain TVL: Failed ({str(e)[:50]})")
    
    # Protocol TVL (Ethereum only)
    for protocol in DEFI_PROTOCOLS:
        try:
            tvl_dict = obj.get_protocol_hist_tvl_by_chain(protocol)
            if 'Ethereum' in tvl_dict:
                df = tvl_dict['Ethereum'].reset_index()
                df = df.rename(columns={'tvl': f'{protocol}_eth_tvl'})
                output_path = os.path.join(OUTPUT_DIR, f'{protocol}_eth_tvl.csv')
                df.to_csv(output_path, index=False)
                print(f"  - {protocol}: {len(df)} days")
            else:
                print(f"  - {protocol}: No Ethereum data")
        except Exception as e:
            print(f"  - {protocol}: Failed ({str(e)[:50]})")
    
    # USDT Market Cap
    try:
        df = obj.get_stablecoin_hist_mcap_on_a_chain(1, 'ethereum')
        df = df.reset_index().rename(columns={'mcap': 'usdt_eth_mcap'})
        output_path = os.path.join(OUTPUT_DIR, 'usdt_eth_mcap.csv')
        df.to_csv(output_path, index=False)
        print(f"  - USDT ETH Mcap: {len(df)} days")
    except Exception as e:
        print(f"  - USDT ETH Mcap: Failed ({str(e)[:50]})")


# ============================================================================
# 6. Layer 2 TVL
# ============================================================================
def collect_l2_tvl():
    """Layer 2 TVL 수집"""
    print(f"\n[6/6] Collecting Layer 2 TVL...")
    
    all_data = []
    
    for chain in L2_CHAINS:
        try:
            url = f"https://api.llama.fi/v2/historicalChainTvl/{chain}"
            response = requests.get(url)
            
            if response.status_code == 200:
                data = response.json()
                df = pd.DataFrame(data)
                df['date'] = pd.to_datetime(df['date'], unit='s')
                df = df.rename(columns={'tvl': f'{chain.lower()}_tvl'})
                all_data.append(df[['date', f'{chain.lower()}_tvl']])
                print(f"  - {chain}: {len(df)} days")
            else:
                print(f"  - {chain}: API error")
        except Exception as e:
            print(f"  - {chain}: Failed ({str(e)[:50]})")
    
    if all_data:
        result = all_data[0]
        for df in all_data[1:]:
            result = pd.merge(result, df, on='date', how='outer')
        
        result = result.sort_values('date').reset_index(drop=True)
        output_path = os.path.join(OUTPUT_DIR, 'layer2_tvl.csv')
        result.to_csv(output_path, index=False)
        print(f"  Saved: {output_path}")


# ============================================================================
# Main Execution
# ============================================================================
if __name__ == "__main__":
    print("=" * 80)
    print("ETH Price Prediction - Data Collection Pipeline")
    print("=" * 80)
    
    collect_upbit_crypto_prices()
    collect_macro_indicators()
    collect_fear_greed()
    collect_funding_rate()
    collect_defi_tvl()
    collect_l2_tvl()
    
    print("\n" + "=" * 80)
    print("Data collection completed!")
    print(f"Output directory: {OUTPUT_DIR}")
    print("=" * 80)




ETH Price Prediction - Data Collection Pipeline

[1/6] Collecting cryptocurrency prices from Upbit...
  - BTC: 2980 days
  - ETH: 2977 days
  - XRP: 2977 days
  - SOL: 1499 days
  - ADA: 2966 days
  - DOGE: 1732 days
  - AVAX: 1383 days
  - DOT: 1874 days
  Saved: ./macro_data/macro_crypto_data.csv

[2/6] Collecting macro indicators...
  - DXY: 2238 days
  - GOLD: 2237 days
  - VIX: 2236 days
  - SP500: 2235 days

[3/6] Collecting Fear & Greed Index...
  - Fear & Greed: 2847 days

[4/6] Collecting ETH funding rate...
  - Funding Rate: 2187 days

[5/6] Collecting DeFi protocol TVL...
  - ETH Chain TVL: 2978 days
  - makerdao: 2515 days
  - lido: 1799 days
  - aave: 2013 days
  - uniswap: 2577 days
  - curve-dex: 2107 days
  - USDT ETH Mcap: 2915 days

[6/6] Collecting Layer 2 TVL...
  - Arbitrum: 1571 days
  - Optimism: 1592 days
  - Base: 891 days
  - zkSync Era: 1290 days
  Saved: ./macro_data/layer2_tvl.csv

Data collection completed!
Output directory: ./macro_data


In [6]:
import os
import time
import warnings
from datetime import datetime, timedelta

import pandas as pd
import requests
import yfinance as yf
from binance.client import Client
from defillama2 import DefiLlama

warnings.filterwarnings('ignore')

def fetch_with_retries(url, max_retries=5, timeout=10, sleep_sec=2):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException:
            if attempt < max_retries - 1:
                time.sleep(sleep_sec)
            else:
                raise

def get_dates():
    today = datetime.today()
    start_date = (today - timedelta(days=200)).strftime('%Y-%m-%d')
    end_date = (today + timedelta(days=1)).strftime('%Y-%m-%d')
    return start_date, end_date

START_DATE, END_DATE = get_dates()
OUTPUT_DIR = f"./macro_data/today/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

UPBIT_CRYPTO_TICKERS = {
    'KRW-BTC': ('BTC', 'BTC'),
    'KRW-ETH': ('ETH', 'ETH'),
    'KRW-XRP': ('XRP', 'XRP'),
    'KRW-SOL': ('SOL', 'SOL'),
    'KRW-ADA': ('ADA', 'ADA'),
    'KRW-DOGE': ('DOGE', 'DOGE'),
    'KRW-AVAX': ('AVAX', 'AVAX'),
    'KRW-DOT': ('DOT', 'DOT')
}

MACRO_TICKERS = {
    'DX-Y.NYB': 'DXY',
    'GC=F': 'GOLD',
    '^VIX': 'VIX',
    '^GSPC': 'SP500'
}

DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']
L2_CHAINS = ['Arbitrum', 'Optimism', 'Base', 'zkSync Era']

def collect_upbit_crypto_prices(start_date):
    start_dt = pd.to_datetime(start_date)
    merged_df = None

    for market, (symbol, _) in UPBIT_CRYPTO_TICKERS.items():
        try:
            all_candles = []
            to_date = None
            while True:
                url = "https://api.upbit.com/v1/candles/days"
                params = {'market': market, 'count': 200}
                if to_date:
                    params['to'] = to_date

                response = requests.get(url, params=params)
                response.raise_for_status()
                candles = response.json()

                if not candles:
                    break

                all_candles.extend(candles)
                oldest_date = pd.to_datetime(candles[-1]['candle_date_time_kst'])
                if oldest_date <= start_dt:
                    break

                to_date = candles[-1]['candle_date_time_utc']
                time.sleep(0.1)

            if not all_candles:
                continue

            df = pd.DataFrame(all_candles)
            df['date'] = pd.to_datetime(df['candle_date_time_kst']).dt.normalize()

            df = df.rename(columns={
                'opening_price': f'{symbol}_Open',
                'high_price': f'{symbol}_High',
                'low_price': f'{symbol}_Low',
                'trade_price': f'{symbol}_Close',
                'candle_acc_trade_volume': f'{symbol}_Volume'
            })

            df = df[['date', f'{symbol}_Open', f'{symbol}_High',
                     f'{symbol}_Low', f'{symbol}_Close', f'{symbol}_Volume']]
            df = df.sort_values('date')
            df = df[df['date'] >= start_dt]

            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on='date', how='outer')

        except Exception:
            continue

    if merged_df is not None:
        merged_df = merged_df.sort_values('date')
        merged_df.to_csv(os.path.join(OUTPUT_DIR, "macro_crypto_data.csv"), index=False)

    return merged_df

def collect_macro_indicators(start_date, end_date):
    for ticker, name in MACRO_TICKERS.items():
        try:
            df = yf.download(ticker, start=start_date, end=end_date, progress=False, group_by='column')
            df = df[['Close']].copy()
            df.columns = [name]
            df.index.name = 'date'
            df.to_csv(os.path.join(OUTPUT_DIR, f"{name}.csv"))
        except Exception:
            continue

def collect_fear_greed():
    try:
        url = "https://api.alternative.me/fng/?limit=4000&format=json"
        data = fetch_with_retries(url)
        df = pd.DataFrame(data['data'])
        df['date'] = pd.to_datetime(df['timestamp'], unit='s')
        df = df[['date', 'value']].rename(columns={'value': 'fear_greed'})
        df = df.sort_values('date').reset_index(drop=True)
        df.to_csv(os.path.join(OUTPUT_DIR, "fear_greed.csv"), index=False)
    except Exception:
        pass

def collect_funding_rate(start_date, end_date):
    try:
        client = Client("", "")
        funding_rates = []

        start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
        end_time = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp() * 1000)
        current_ts = start_time

        while current_ts < end_time:
            rates = client.futures_funding_rate(symbol='ETHUSDT', startTime=current_ts, limit=1000)
            if not rates:
                break
            funding_rates.extend(rates)
            current_ts = rates[-1]['fundingTime'] + 1

        df = pd.DataFrame(funding_rates)
        df['date'] = pd.to_datetime(df['fundingTime'], unit='ms').dt.date
        df['fundingRate'] = df['fundingRate'].astype(float)
        df = df.groupby('date')['fundingRate'].mean().reset_index()
        df.to_csv(os.path.join(OUTPUT_DIR, "eth_funding_rate.csv"), index=False)
    except Exception:
        pass

def collect_defi_tvl():
    obj = DefiLlama()
    try:
        df = obj.get_chain_hist_tvl('Ethereum')
        df = df.reset_index().rename(columns={'tvl': 'eth_chain_tvl'})
        df.to_csv(os.path.join(OUTPUT_DIR, 'eth_chain_tvl.csv'), index=False)
    except Exception:
        pass

    for protocol in DEFI_PROTOCOLS:
        try:
            attempts = 5
            for i in range(attempts):
                try:
                    tvl_dict = obj.get_protocol_hist_tvl_by_chain(protocol)
                    if 'Ethereum' in tvl_dict:
                        df = tvl_dict['Ethereum'].reset_index()
                        df = df.rename(columns={'tvl': f'{protocol}_eth_tvl'})
                        df.to_csv(os.path.join(OUTPUT_DIR, f'{protocol}_eth_tvl.csv'), index=False)
                    break
                except Exception:
                    if i == attempts - 1:
                        raise
                    time.sleep(2)
        except Exception:
            continue

    try:
        df = obj.get_stablecoin_hist_mcap_on_a_chain(1, 'ethereum')
        df = df.reset_index().rename(columns={'mcap': 'usdt_eth_mcap'})
        df.to_csv(os.path.join(OUTPUT_DIR, 'usdt_eth_mcap.csv'), index=False)
    except Exception:
        pass

def collect_l2_tvl():
    all_data = []
    for chain in L2_CHAINS:
        try:
            attempts = 5
            for i in range(attempts):
                try:
                    url = f"https://api.llama.fi/v2/historicalChainTvl/{chain}"
                    data = fetch_with_retries(url)
                    df = pd.DataFrame(data)
                    df['date'] = pd.to_datetime(df['date'], unit='s')
                    df = df.rename(columns={'tvl': f'{chain.lower()}_tvl'})
                    all_data.append(df[['date', f'{chain.lower()}_tvl']])
                    break
                except Exception:
                    if i == attempts - 1:
                        raise
                    time.sleep(2)
        except Exception:
            continue

    if all_data:
        result = all_data[0]
        for df in all_data[1:]:
            result = pd.merge(result, df, on='date', how='outer')

        result = result.sort_values('date').reset_index(drop=True)
        result.to_csv(os.path.join(OUTPUT_DIR, 'layer2_tvl.csv'), index=False)

if __name__ == "__main__":
    collect_upbit_crypto_prices(START_DATE)
    collect_macro_indicators(START_DATE, END_DATE)
    collect_fear_greed()
    collect_funding_rate(START_DATE, END_DATE)
    collect_defi_tvl()
    collect_l2_tvl()


In [ ]:
import os
import time
import warnings
from datetime import datetime, timedelta
import pandas as pd
import requests
import yfinance as yf
from defillama2 import DefiLlama
from binance.client import Client

warnings.filterwarnings('ignore')

# ============================================================================
# 설정 (Configuration)
# ============================================================================
START_DATE = "2017-01-01"
END_DATE = datetime.now().strftime('%Y-%m-%d')
OUTPUT_DIR = "./dataset"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 업비트 4시간봉 (240분)
TIMEFRAME_MINUTES = 240 

UPBIT_CRYPTO_TICKERS = {
    'KRW-BTC': 'BTC',
    'KRW-ETH': 'ETH',
    'KRW-XRP': 'XRP',
    'KRW-SOL': 'SOL',
    'KRW-ADA': 'ADA',
    'KRW-DOGE': 'DOGE'
}

MACRO_TICKERS = {
    'DX-Y.NYB': 'DXY',
    'GC=F': 'GOLD',
    '^VIX': 'VIX',
    '^GSPC': 'SP500',
    '^IXIC': 'NASDAQ'
}

DEFI_PROTOCOLS = ['makerdao', 'lido', 'aave', 'uniswap', 'curve-dex']

# ============================================================================
# 1. 업비트 4시간봉 수집 (KST 기준)
# ============================================================================
def collect_upbit_4h_candles():
    print(f"\n[1/6] Collecting Upbit 4-Hour Candles (KST Base)...")
    
    start_dt = pd.to_datetime(START_DATE)
    merged_df = None
    
    for market, symbol in UPBIT_CRYPTO_TICKERS.items():
        try:
            all_candles = []
            to_date = None
            
            while True:
                url = f"https://api.upbit.com/v1/candles/minutes/{TIMEFRAME_MINUTES}"
                params = {'market': market, 'count': 200}
                if to_date:
                    params['to'] = to_date
                
                resp = requests.get(url, params=params)
                if resp.status_code != 200:
                    time.sleep(1)
                    continue
                    
                candles = resp.json()
                if not candles:
                    break
                
                all_candles.extend(candles)
                
                # KST 시간 기준 확인
                last_dt_kst = pd.to_datetime(candles[-1]['candle_date_time_kst'])
                
                if last_dt_kst < start_dt:
                    break
                
                to_date = candles[-1]['candle_date_time_utc']
                time.sleep(0.1)
            
            if not all_candles:
                continue
                
            df = pd.DataFrame(all_candles)
            # [중요] KST 시간 사용
            df['datetime'] = pd.to_datetime(df['candle_date_time_kst'])
            
            cols = {
                'opening_price': f'{symbol}_Open',
                'high_price': f'{symbol}_High',
                'low_price': f'{symbol}_Low',
                'trade_price': f'{symbol}_Close',
                'candle_acc_trade_volume': f'{symbol}_Volume'
            }
            df = df.rename(columns=cols)[['datetime'] + list(cols.values())]
            
            # 정렬 및 중복 제거
            df = df.sort_values('datetime').drop_duplicates('datetime')
            df = df[df['datetime'] >= start_dt]
            
            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on='datetime', how='outer')
                
            print(f"  - {symbol}: {len(df)} rows collected")
            
        except Exception as e:
            print(f"  - {symbol} Error: {e}")
    
    # 4시간 정각 인덱스 리샘플링 (중간 누락 채우기)
    if merged_df is not None:
        merged_df = merged_df.sort_values('datetime').set_index('datetime')
        merged_df = merged_df.resample('4H').first() 
        merged_df = merged_df.reset_index()
        
    return merged_df

# ============================================================================
# 2. 거시/온체인 일봉 데이터 (KST 09:00 기준 매핑용)
# ============================================================================
def collect_daily_metrics():
    print(f"\n[2/6] Collecting Daily Metrics (Fees, Macro, TVL)...")
    daily_data_map = {}
    
    # 2-1. Macro
    for ticker, name in MACRO_TICKERS.items():
        try:
            df = yf.download(ticker, start=START_DATE, progress=False)
            if isinstance(df.columns, pd.MultiIndex): df = df['Close']
            else: df = df[['Close']]
            
            df.columns = [name]
            df.index = pd.to_datetime(df.index).normalize()
            daily_data_map[name] = df
        except: pass

    # 2-2. DefiLlama (TVL & Fees)
    obj = DefiLlama()
    
    # Fees (우리가 중요하게 여긴 데이터)
    for proto in DEFI_PROTOCOLS:
        try:
            # 1. TVL
            tvl = obj.get_protocol_hist_tvl_by_chain(proto)
            if 'Ethereum' in tvl:
                df_tvl = tvl['Ethereum']
                df_tvl.index = pd.to_datetime(df_tvl.index).normalize()
                df_tvl.columns = [f'{proto}_tvl']
                daily_data_map[f'{proto}_tvl'] = df_tvl
            
            # 2. Fees (Revenue) - 핵심 데이터
            url = f"https://api.llama.fi/summary/fees/{proto}?dataType=dailyFees"
            res = requests.get(url).json()
            if 'totalDataChart' in res:
                df_fees = pd.DataFrame(res['totalDataChart'])
                # Unix Timestamp -> Date
                df_fees['date'] = pd.to_datetime(df_fees['date'], unit='s').dt.normalize()
                df_fees = df_fees.set_index('date')[['value']].rename(columns={'value': f'{proto}_fees'})
                daily_data_map[f'{proto}_fees'] = df_fees
                print(f"  - {proto} Fees collected")
        except: pass
        
    return daily_data_map

# ============================================================================
# 3. [추가됨] Binance Open Interest & Funding Rate (4H)
# ============================================================================
def collect_binance_oi_funding():
    print(f"\n[3/6] Collecting Binance OI & Funding Rate...")
    
    try:
        client = Client() 
        
        # 1. Funding Rate (8시간 단위)
        # API 제한 때문에 최근 1000개만 가져옴 (실전용 루프 생략)
        fr = client.futures_funding_rate(symbol='ETHUSDT', limit=1000)
        df_fr = pd.DataFrame(fr)
        df_fr['datetime'] = pd.to_datetime(df_fr['fundingTime'], unit='ms')
        df_fr['fundingRate'] = df_fr['fundingRate'].astype(float)
        
        # KST 변환 (UTC+9)
        df_fr['datetime'] = df_fr['datetime'] + timedelta(hours=9)
        
        # 4H Resample
        df_fr = df_fr.set_index('datetime').resample('4H').ffill()
        df_fr = df_fr.rename(columns={'fundingRate': 'eth_funding_rate'})
        
        # 2. Open Interest (Hist) - 핵심 데이터
        # 바이낸스는 'futures_open_interest_hist'로 5분~1시간 단위 제공
        # 최근 데이터(최대 30일 등)만 제공될 수 있음.
        oi = client.futures_open_interest_hist(symbol='ETHUSDT', period='4h', limit=500)
        df_oi = pd.DataFrame(oi)
        df_oi['datetime'] = pd.to_datetime(df_oi['timestamp'], unit='ms')
        
        # KST 변환
        df_oi['datetime'] = df_oi['datetime'] + timedelta(hours=9)
        
        df_oi['sumOpenInterest'] = df_oi['sumOpenInterest'].astype(float)
        df_oi['sumOpenInterestValue'] = df_oi['sumOpenInterestValue'].astype(float)
        
        # 4H Resample (이미 4h지만 인덱스 맞춤)
        df_oi = df_oi.set_index('datetime').resample('4H').last()
        df_oi = df_oi[['sumOpenInterest', 'sumOpenInterestValue']]
        df_oi.columns = ['eth_oi_token', 'eth_oi_usdt']
        
        # Merge FR + OI
        df_binance = df_fr.join(df_oi, how='outer')
        
        print(f"  - Binance OI & Funding collected")
        return df_binance
        
    except Exception as e:
        print(f"  ! Binance Error: {e}")
        return pd.DataFrame()

# ============================================================================
# 4. 병합 (Merge & Time Alignment)
# ============================================================================
def merge_all_data(base_df, daily_map, binance_df):
    print(f"\n[4/6] Merging Datasets...")
    
    final_df = base_df.copy()
    final_df = final_df.sort_index()
    
    # 1. 일봉 데이터 병합 (Shift 적용: 어제 데이터 -> 오늘 사용)
    # 업비트 KST 09:00 캔들 = 전일 종가 확정 직후
    # 따라서 전일(T-1) 일봉 데이터를 T일 09:00 이후 캔들에 매핑
    
    # Join Key: '날짜' (시간 제외)
    # 업비트 캔들의 날짜에서 1일을 뺌 -> 그 날짜의 일봉 데이터를 찾음
    final_df['prev_date'] = final_df['datetime'].dt.normalize() - timedelta(days=1)
    
    for name, df_daily in daily_map.items():
        temp = df_daily.reset_index()
        temp.columns = ['prev_date', name]
        final_df = pd.merge(final_df, temp, on='prev_date', how='left')
        final_df[name] = final_df[name].fillna(method='ffill')
        
    final_df = final_df.drop(columns=['prev_date'])
    
    # 2. 바이낸스 데이터 병합 (같은 4시간봉끼리 매칭)
    if not binance_df.empty:
        final_df = final_df.set_index('datetime')
        final_df = final_df.join(binance_df, how='left')
        final_df = final_df.reset_index()
        
        # 결측치 처리 (OI는 민감하므로 ffill 조심해야 하나, 데이터 끊김 방지용으로 사용)
        final_df['eth_funding_rate'] = final_df['eth_funding_rate'].fillna(method='ffill')
        final_df['eth_oi_usdt'] = final_df['eth_oi_usdt'].fillna(method='ffill')
        
    return final_df

# ============================================================================
# Main
# ============================================================================
if __name__ == "__main__":
    # 1. Base
    df_base = collect_upbit_4h_candles()
    
    if df_base is not None:
        # 2. Daily
        daily_data = collect_daily_metrics()
        
        # 3. Binance (OI + FR)
        binance_data = collect_binance_oi_funding()
        
        # 4. Merge
        final_df = merge_all_data(df_base, daily_data, binance_data)
        
        # 5. Save
        final_df = final_df.dropna(subset=['ETH_Close'])
        final_df = final_df.fillna(method='ffill').fillna(method='bfill')
        
        save_path = os.path.join(OUTPUT_DIR, "eth_4hour.csv")
        final_df.to_csv(save_path, index=False)
        
        print(f"\n[Complete] Saved to {save_path}")
        print(f"Columns: {final_df.columns.tolist()}")
